In [ ]:
import pandas as pd
import requests

url = "http://127.0.0.1:8000/api/journal/"
data = requests.get(url).json()

df = pd.DataFrame(data)
df[['symptom_text']].head()


,symptom_text
0,Feeling normal overall but slightly tired afte...
1,"Cold sensation with fever and headache, diffic..."
2,General weakness and fatigue after long workin...
3,"Feeling stressed due to exams, heart rate feel..."
4,Shortness of breath when climbing stairs and f...


In [ ]:
def label_text(text):
    text = text.lower()
    if "fever" in text or "temperature" in text:
        return "fever"
    elif "chest" in text or "breath" in text:
        return "cardio"
    elif "tired" in text or "fatigue" in text:
        return "fatigue"
    else:
        return "general"

df['label'] = df['symptom_text'].apply(label_text)
df[['symptom_text', 'label']].head()


,symptom_text,label
0,Feeling normal overall but slightly tired afte...,fatigue
1,"Cold sensation with fever and headache, diffic...",fever
2,General weakness and fatigue after long workin...,fatigue
3,"Feeling stressed due to exams, heart rate feel...",general
4,Shortness of breath when climbing stairs and f...,cardio


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

X = df['symptom_text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('clf', MultinomialNB())
])

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, zero_division=0))


Accuracy: 0.6
              precision    recall  f1-score   support

     fatigue       0.00      0.00      0.00         2
       fever       0.67      1.00      0.80         2
     general       0.50      1.00      0.67         1

    accuracy                           0.60         5
   macro avg       0.39      0.67      0.49         5
weighted avg       0.37      0.60      0.45         5



In [ ]:
test_sentences = [
    "Having fever and headache since last night",
    "Chest pain and breathing problem while walking",
    "Feeling very tired after exams"
]

model.predict(test_sentences)


array(['fever', 'cardio', 'general'], dtype='<U7')

In [ ]:
def late_fusion(sensor_pred, text_label,
                sensor_weight=0.6, text_weight=0.4):
    
    # Map NLP labels to risk scores
    text_risk_map = {
        "general": 0.2,
        "fatigue": 0.4,
        "fever": 0.7,
        "cardio": 0.8
    }
    
    sensor_score = 1.0 if sensor_pred == 1 else 0.0
    text_score = text_risk_map.get(text_label, 0.3)
    
    final_score = (sensor_weight * sensor_score) + \
                  (text_weight * text_score)
    
    return round(final_score, 2)

In [ ]:
# Example test cases
print(late_fusion(1, "fever"))     # high risk
print(late_fusion(0, "fatigue"))   # moderate / low risk
print(late_fusion(1, "cardio"))    # very high risk


0.88
0.16
0.92


In [ ]:
def get_recommendation(risk_level, symptom_label):
    recommendations = []

    # General advice for all users
    recommendations.append("Maintain hydration")
    recommendations.append("Get adequate rest")

    if risk_level == "High Risk":
        recommendations.append("Consult a healthcare professional")
        recommendations.append("Avoid physical exertion")

        if symptom_label == "cardio":
            recommendations.append("Avoid strenuous activities")
            recommendations.append("Monitor heart rate regularly")

        elif symptom_label == "fever":
            recommendations.append("Monitor body temperature")
            recommendations.append("Take rest and light meals")

    elif risk_level == "Moderate Risk":
        recommendations.append("Light physical activity like walking")

        if symptom_label == "fatigue":
            recommendations.append("Improve sleep schedule")

        elif symptom_label == "fever":
            recommendations.append("Increase fluid intake")

    else:  # Low Risk
        recommendations.append("Continue normal daily activities")
        recommendations.append("Maintain balanced diet")

    return recommendations
